### Çalışmanın Amacı : Burma ülkesinde faaliyetini sürdüren bir istatistik ve reklam ajansı olası üretici, dağıtıcı ve parakende satıcı müşterilerine bilgi sağlayabilmek.
### Veri Seti : Ülke genelinde faaliyet gösteren süpermarket zincirinin, ülkedeki en büyük 3 şehirdeki 1000 satışlık verileri. 

In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [121]:
pd.read_csv("supermarket_sales.csv", sep=",", header =0 , index_col = 0 , decimal=".")#sep = "," veriler , ile ayrılıyor // header = 0 sütunların başlıkları başlangıç satırında. // index_col = indexin hangi collumnda olduğunu tanımlar. // skiprows = 5 ilk 5 satırdaki verileri alma demek // decimal = "." ondalık nokta ile ayrılıyor.

,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
Invoice ID,,,,,,,,,,,,,,,,
750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7


In [122]:
df = df = pd.read_csv("supermarket_sales.csv", sep=",", header =0 , index_col = 0 , decimal=".")

In [123]:
print(df.describe())

        Unit price     Quantity       Tax 5%        Total        cogs   
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.00000  \
mean     55.672130     5.510000    15.379369   322.966749   307.58738   
std      26.494628     2.923431    11.708825   245.885335   234.17651   
min      10.080000     1.000000     0.508500    10.678500    10.17000   
25%      32.875000     3.000000     5.924875   124.422375   118.49750   
50%      55.230000     5.000000    12.088000   253.848000   241.76000   
75%      77.935000     8.000000    22.445250   471.350250   448.90500   
max      99.960000    10.000000    49.650000  1042.650000   993.00000   

       gross margin percentage  gross income      Rating  
count              1000.000000   1000.000000  1000.00000  
mean                  4.761905     15.379369     6.97270  
std                   0.000000     11.708825     1.71858  
min                   4.761905      0.508500     4.00000  
25%                   4.761905      5.924875   

In [124]:
df = df.drop(columns=['Date', 'Time','Branch' ])
# Date,Time ve Branch columnlarını sildik, Date ve Time columnlarımız karışık yazılmıştı ve makine öğrenim aşamasında hatalarla karşı karşıya gelebilirdi.
# Branch columnu ise City columnu ile %100 lük eşleşmeye sahip olduğu için 1.0 doğruluk oranına sahip olacaktık.

Denetimli Öğrenme "Random Forest" sınıflandırma modeli kullanımı:

In [100]:
le = LabelEncoder()
categorical_features = ['City', 'Customer type', 'Gender', 'Product line', 'Payment']
for col in categorical_features:
    df[col] = le.fit_transform(df[col])
X = df.drop(columns=['Customer type'])
y = df['Customer type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Model Customer type tahminlerinin ", accuracy_score(y_test, y_pred),"kadarını doğru tahmin etti")

Model Customer type tahminlerinin  0.49 kadarını doğru tahmin etti


In [101]:
le = LabelEncoder()
categorical_features = ['City', 'Customer type', 'Gender', 'Product line', 'Payment']
for col in categorical_features:
    df[col] = le.fit_transform(df[col])
X = df.drop(columns=['City'])
y = df['City']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Model City tahminlerinin ", accuracy_score(y_test, y_pred),"kadarını doğru tahmin etti")

Model City tahminlerinin  0.275 kadarını doğru tahmin etti


In [102]:
le = LabelEncoder()
categorical_features = [ 'City', 'Customer type', 'Gender', 'Product line', 'Payment']
for col in categorical_features:
    df[col] = le.fit_transform(df[col])
X = df.drop(columns=['Gender'])
y = df['Gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Model Gender tahminlerinin ", accuracy_score(y_test, y_pred),"kadarını doğru tahmin etti")

Model Gender tahminlerinin  0.495 kadarını doğru tahmin etti


In [125]:
le = LabelEncoder()
categorical_features = [ 'City', 'Customer type', 'Gender', 'Product line', 'Payment']
for col in categorical_features:
    df[col] = le.fit_transform(df[col])
X = df.drop(columns=['Product line'])
y = df['Product line']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Model Product line tahminlerinin ", accuracy_score(y_test, y_pred),"kadarını doğru tahmin etti")

Model Product line tahminlerinin  0.185 kadarını doğru tahmin etti


In [126]:
le = LabelEncoder()
categorical_features = [ 'City', 'Customer type', 'Gender', 'Product line', 'Payment']
for col in categorical_features:
    df[col] = le.fit_transform(df[col])
X = df.drop(columns=['Payment'])
y = df['Payment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Model Payment tahminlerinin ", accuracy_score(y_test, y_pred),"kadarını doğru tahmin etti")

Model Payment tahminlerinin  0.31 kadarını doğru tahmin etti


Çapraz Doğrulama modeli:

In [127]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print("Çapraz Doğrulama Skoru: ", scores)
print("Ortalama Çapraz Doğrulama Skoru: ", scores.mean())


Çapraz Doğrulama Skoru:  [0.325 0.335 0.31  0.28  0.29 ]
Ortalama Çapraz Doğrulama Skoru:  0.308


Hiperparametre Ayarlaması:

In [106]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 150, 200],  
    'max_depth': [None, 10, 20, 30], 
    'min_samples_split': [2, 5, 10] 
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
model = grid_search.best_estimator_
print("Her bir karar ağacının maksimum derinliği 20'dir. Herhangi bir düğümün bölünebilmesi için en az 2 örneğe sahip olması gereklidir. Modelimiz toplamda 150 karar ağacı oluşturacaktır.")

{'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 150}
Her bir karar ağacının maksimum derinliği 20'dir. Herhangi bir düğümün bölünebilmesi için en az 2 örneğe sahip olması gereklidir. Modelimiz toplamda 150 karar ağacı oluşturacaktır.


In [107]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Doğruluğu: ", accuracy ,)

Test Doğruluğu:  0.295


Destek Vektör Makineleri modeli denemesi:

In [128]:
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Doğruluğu: ", accuracy)
from sklearn.model_selection import GridSearchCV

parameters = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  

grid = GridSearchCV(svm.SVC(), parameters, refit = True, verbose = 3)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)


Test Doğruluğu:  0.29
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.344 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.344 total time=   0.0s
[CV 2/5] END .....C=0.1, 

Karar Ağacı denemesi:

In [113]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: ", accuracy)

Test Accuracy:  0.315


Lojistik Regresyon Denemesi:

In [138]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: ", accuracy)

Test Accuracy:  0.3


### Yapılan işlem : Gereksiz attributeların silinmesi

In [116]:
df = pd.read_csv("supermarket_sales.csv", sep=",", header =0 , index_col = 0 , decimal=".")
df = df.drop('Branch', axis=1)
df = df.drop("Tax 5%", axis=1)
df = df.drop("Date", axis=1)
df = df.drop("Time", axis=1)
df = df.drop("Rating", axis=1)
df = df.drop("cogs", axis=1)
df = df.drop("gross income", axis=1)
df = df.drop("gross margin percentage", axis=1)
print(df)

                  City Customer type  Gender            Product line   
Invoice ID                                                             
750-67-8428     Yangon        Member  Female       Health and beauty  \
226-31-3081  Naypyitaw        Normal  Female  Electronic accessories   
631-41-3108     Yangon        Normal    Male      Home and lifestyle   
123-19-1176     Yangon        Member    Male       Health and beauty   
373-73-7910     Yangon        Normal    Male       Sports and travel   
...                ...           ...     ...                     ...   
233-67-5758  Naypyitaw        Normal    Male       Health and beauty   
303-96-2227   Mandalay        Normal  Female      Home and lifestyle   
727-02-1313     Yangon        Member    Male      Food and beverages   
347-56-2442     Yangon        Normal    Male      Home and lifestyle   
849-09-3807     Yangon        Member  Female     Fashion accessories   

             Unit price  Quantity      Total      Payment  
Inv

In [117]:
df.to_csv('Düzenlenmis_supermarket_sales.csv', index=True)

### Yapılan İşlem : Eksik veri kontrolü


In [17]:
missing_values = df.isnull().sum()
print(missing_values)

City             0
Customer type    0
Gender           0
Product line     0
Unit price       0
Quantity         0
Total            0
Payment          0
dtype: int64


### Yapılan işlem : Toplam şehir sayısı ve isimleri

In [18]:
unique_cities = df["City"].unique()
unique_cities_quantity = df["City"].nunique()
print(unique_cities)
print(unique_cities_quantity)

['Yangon' 'Naypyitaw' 'Mandalay']
3


### Yapılan işlem : Toplam "Product Line" sayısı ve İsimleri

In [19]:
unique_product_line = df["Product line"].unique()
unique_product_line_quantity = df["Product line"].nunique()
print(unique_product_line)
print(unique_product_line_quantity)

['Health and beauty' 'Electronic accessories' 'Home and lifestyle'
 'Sports and travel' 'Food and beverages' 'Fashion accessories']
6


In [20]:
total_product_line_tendency = df['Product line'].value_counts(normalize=True)
print("Ülke geneli product line satın alma eğilimi:")
print(total_product_line_tendency)

Ülke geneli product line satın alma eğilimi:
Product line
Fashion accessories       0.178
Food and beverages        0.174
Electronic accessories    0.170
Sports and travel         0.166
Home and lifestyle        0.160
Health and beauty         0.152
Name: proportion, dtype: float64


In [10]:
product_lines1 = ['Health and beauty', 'Electronic accessories', 'Home and lifestyle']
for product in product_lines1:
    print(f"\n{product} için şehirlerdeki para harcama eğilimi:")
    spending_by_city = df[df['Product line'] == product].groupby('City')['Total'].sum()
    spending_by_city /= spending_by_city.sum()
    print(spending_by_city.sort_values(ascending=False))


Health and beauty için şehirlerdeki para harcama eğilimi:
City
Mandalay     0.406163
Naypyitaw    0.337753
Yangon       0.256084
Name: Total, dtype: float64

Electronic accessories için şehirlerdeki para harcama eğilimi:
City
Naypyitaw    0.349095
Yangon       0.337099
Mandalay     0.313806
Name: Total, dtype: float64

Home and lifestyle için şehirlerdeki para harcama eğilimi:
City
Yangon       0.416198
Mandalay     0.325818
Naypyitaw    0.257985
Name: Total, dtype: float64


In [11]:
product_lines2 = ['Sports and travel', 'Food and beverages', 'Fashion accessories']
for product in product_lines2:
    print(f"\n{product} için şehirlerdeki para harcama eğilimi:")
    spending_by_city = df[df['Product line'] == product].groupby('City')['Total'].sum()
    spending_by_city /= spending_by_city.sum()
    print(spending_by_city.sort_values(ascending=False))


Sports and travel için şehirlerdeki para harcama eğilimi:
City
Mandalay     0.362612
Yangon       0.351446
Naypyitaw    0.285942
Name: Total, dtype: float64

Food and beverages için şehirlerdeki para harcama eğilimi:
City
Naypyitaw    0.423313
Yangon       0.305693
Mandalay     0.270994
Name: Total, dtype: float64

Fashion accessories için şehirlerdeki para harcama eğilimi:
City
Naypyitaw    0.397012
Mandalay     0.302238
Yangon       0.300750
Name: Total, dtype: float64


### Yapılan İşlem : Bir kadın müşterinin mağazaya girdiğinde hangi kategöriden alışveriş yapma eğilimi olduğunun bulunması.

In [12]:
female_health_and_beauty = df[(df['Gender'] == 'Female') & (df['Product line'] == 'Health and beauty')]
female_other_product_lines = df[(df['Gender'] == 'Female') & (df['Product line'] != 'Health and beauty')]
intersection = len(female_health_and_beauty)
union = len(female_health_and_beauty) + len(female_other_product_lines)
female_health_and_beauty_egilim = intersection / union
print("Kadınların Health and beauty kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * female_health_and_beauty_egilim))
female_Electronic_accessories = df[(df['Gender'] == 'Female') & (df['Product line'] == 'Electronic accessories')]
female_other_product_lines = df[(df['Gender'] == 'Female') & (df['Product line'] != 'Electronic accessories')]
intersection = len(female_Electronic_accessories)
union = len(female_Electronic_accessories) + len(female_other_product_lines)
female_Electronic_accessories_egilim = intersection / union
print("Kadınların Electronic accessories kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * female_Electronic_accessories_egilim))
female_Home_lifestyle = df[(df['Gender'] == 'Female') & (df['Product line'] == 'Home and lifestyle')]
female_other_product_lines = df[(df['Gender'] == 'Female') & (df['Product line'] != 'Home and lifestyle')]
intersection = len(female_Home_lifestyle)
union = len(female_Home_lifestyle) + len(female_other_product_lines)
female_Home_lifestyle_egilim = intersection / union
print("Kadınların Home and lifestyle kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * female_Home_lifestyle_egilim))
female_Sports_travel = df[(df['Gender'] == 'Female') & (df['Product line'] == 'Sports and travel')]
female_other_product_lines = df[(df['Gender'] == 'Female') & (df['Product line'] != 'Sports and travel')]
intersection = len(female_Sports_travel)
union = len(female_Sports_travel) + len(female_other_product_lines)
female_Sports_travel_egilim = intersection / union
print("Kadınların Sports and Travel kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * female_Sports_travel_egilim))
female_Food_beverages = df[(df['Gender'] == 'Female') & (df['Product line'] == 'Food and beverages')]
female_other_product_lines = df[(df['Gender'] == 'Female') & (df['Product line'] != 'Food and beverages')]
intersection = len(female_Food_beverages)
union = len(female_Food_beverages) + len(female_other_product_lines)
female_Food_beverages_egilim = intersection / union
print("Kadınların Health and beauty kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * female_Food_beverages_egilim))
female_Fashion_accessories = df[(df['Gender'] == 'Female') & (df['Product line'] == 'Fashion accessories')]
female_other_product_lines = df[(df['Gender'] == 'Female') & (df['Product line'] != 'Fashion accessories')]
intersection = len(female_Fashion_accessories)
union = len(female_Fashion_accessories) + len(female_other_product_lines)
female_Fashion_accessories_egilim = intersection / union
print("Kadınların Health and beauty kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * female_Fashion_accessories_egilim))
female_egilimler = {
    'Electronic accessories': female_Electronic_accessories_egilim,
    'Health and beauty': female_health_and_beauty_egilim,
    'Home and lifestyle': female_Home_lifestyle_egilim,
    'Sports and travel': female_Sports_travel_egilim,
    'Food and beverages': female_Food_beverages_egilim,
    'Fashion accessories': female_Fashion_accessories_egilim
}
df_female_egilimler = pd.DataFrame(list(female_egilimler.items()), columns=['Product line', 'Eğilim'])
df_female_egilimler = df_female_egilimler.sort_values('Eğilim', ascending=False)
df_female_egilimler = df_female_egilimler.set_index('Product line')
print(df_female_egilimler)



Kadınların Health and beauty kategorisinde alışveriş yapma eğilimi :  % 12.77445109780439
Kadınların Electronic accessories kategorisinde alışveriş yapma eğilimi :  % 16.766467065868262
Kadınların Home and lifestyle kategorisinde alışveriş yapma eğilimi :  % 15.768463073852296
Kadınların Sports and Travel kategorisinde alışveriş yapma eğilimi :  % 17.56487025948104
Kadınların Health and beauty kategorisinde alışveriş yapma eğilimi :  % 17.964071856287426
Kadınların Health and beauty kategorisinde alışveriş yapma eğilimi :  % 19.16167664670659
                          Eğilim
Product line                    
Fashion accessories     0.191617
Food and beverages      0.179641
Sports and travel       0.175649
Electronic accessories  0.167665
Home and lifestyle      0.157685
Health and beauty       0.127745


### Yapılan İşlem : Bir erkek müşterinin mağazaya girdiğinde hangi kategöriden alışveriş yapma eğilimi olduğunun bulunması.

In [13]:
Male_health_and_beauty = df[(df['Gender'] == 'Male') & (df['Product line'] == 'Health and beauty')]
Male_other_product_lines = df[(df['Gender'] == 'Male') & (df['Product line'] != 'Health and beauty')]
intersection = len(Male_health_and_beauty)
union = len(Male_health_and_beauty) + len(Male_other_product_lines)
Male_health_and_beauty_egilim = intersection / union
print("Erkeklerin Health and beauty kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * Male_health_and_beauty_egilim))
Male_Electronic_accessories = df[(df['Gender'] == 'Male') & (df['Product line'] == 'Electronic accessories')]
Male_other_product_lines = df[(df['Gender'] == 'Male') & (df['Product line'] != 'Electronic accessories')]
intersection = len(Male_Electronic_accessories)
union = len(Male_Electronic_accessories) + len(Male_other_product_lines)
Male_Electronic_accessories_egilim = intersection / union
print("Erkeklerin Electronic accessories kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * Male_Electronic_accessories_egilim))
Male_Home_lifestyle = df[(df['Gender'] == 'Male') & (df['Product line'] == 'Home and lifestyle')]
Male_other_product_lines = df[(df['Gender'] == 'Male') & (df['Product line'] != 'Home and lifestyle')]
intersection = len(Male_Home_lifestyle)
union = len(Male_Home_lifestyle) + len(Male_other_product_lines)
Male_Home_lifestyle_egilim = intersection / union
print("Erkeklerin Home and lifestyle kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * Male_Home_lifestyle_egilim))
Male_Sports_travel = df[(df['Gender'] == 'Male') & (df['Product line'] == 'Sports and travel')]
Male_other_product_lines = df[(df['Gender'] == 'Male') & (df['Product line'] != 'Sports and travel')]
intersection = len(Male_Sports_travel)
union = len(Male_Sports_travel) + len(Male_other_product_lines)
Male_Sports_travel_egilim = intersection / union
print("Erkeklerin Sports and Travel kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * Male_Sports_travel_egilim))
Male_Food_beverages = df[(df['Gender'] == 'Male') & (df['Product line'] == 'Food and beverages')]
Male_other_product_lines = df[(df['Gender'] == 'Male') & (df['Product line'] != 'Food and beverages')]
intersection = len(Male_Food_beverages)
union = len(Male_Food_beverages) + len(Male_other_product_lines)
Male_Food_beverages_egilim = intersection / union
print("Erkeklerin Food and beverages kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * Male_Food_beverages_egilim))
Male_Fashion_accessories = df[(df['Gender'] == 'Male') & (df['Product line'] == 'Fashion accessories')]
Male_other_product_lines = df[(df['Gender'] == 'Male') & (df['Product line'] != 'Fashion accessories')]
intersection = len(Male_Fashion_accessories)
union = len(Male_Fashion_accessories) + len(Male_other_product_lines)
Male_Fashion_accessories_egilim = intersection / union
print("Erkeklerin Fashion and accessories kategorisinde alışveriş yapma eğilimi : " , "%" ,(100 * Male_Fashion_accessories_egilim))
Male_egilimler = {
    'Electronic accessories': Male_Electronic_accessories_egilim,
    'Health and beauty': Male_health_and_beauty_egilim,
    'Home and lifestyle': Male_Home_lifestyle_egilim,
    'Sports and travel': Male_Sports_travel_egilim,
    'Food and beverages': Male_Food_beverages_egilim,
    'Fashion accessories': Male_Fashion_accessories_egilim
}
df_Male_egilimler = pd.DataFrame(list(Male_egilimler.items()), columns=['Product line', 'Eğilim'])
df_Male_egilimler = df_Male_egilimler.sort_values('Eğilim', ascending=False)
df_Male_egilimler = df_Male_egilimler.set_index('Product line')
print(df_Male_egilimler)

Erkeklerin Health and beauty kategorisinde alışveriş yapma eğilimi :  % 17.635270541082164
Erkeklerin Electronic accessories kategorisinde alışveriş yapma eğilimi :  % 17.234468937875754
Erkeklerin Home and lifestyle kategorisinde alışveriş yapma eğilimi :  % 16.23246492985972
Erkeklerin Sports and Travel kategorisinde alışveriş yapma eğilimi :  % 15.631262525050099
Erkeklerin Food and beverages kategorisinde alışveriş yapma eğilimi :  % 16.83366733466934
Erkeklerin Fashion and accessories kategorisinde alışveriş yapma eğilimi :  % 16.432865731462925
                          Eğilim
Product line                    
Health and beauty       0.176353
Electronic accessories  0.172345
Food and beverages      0.168337
Fashion accessories     0.164329
Home and lifestyle      0.162325
Sports and travel       0.156313


 * ### Yukarıda yapılan kategoriye göre eğilim hesaplamalarında, bir cinsiyete bağlı olarak müşterinin hangi kategöride alışveriş yapma eğilimi olduğunu ortaya çıkartmıştır. 
 * ### Hesaplamaya harcanan para girmemiştir.

In [14]:
total_spending_per_product_line = df.groupby('Product line')['Total'].sum()
total_spending_per_product_line /= total_spending_per_product_line.sum()
print("Ülke genelindeki product line para harcama eğilimi:")
print(total_spending_per_product_line.sort_values(ascending=False))
print("\n")

Ülke genelindeki product line para harcama eğilimi:
Product line
Food and beverages        0.173841
Sports and travel         0.170676
Electronic accessories    0.168245
Fashion accessories       0.168147
Home and lifestyle        0.166772
Health and beauty         0.152318
Name: Total, dtype: float64




In [15]:
print("Ülke genelindeki cinsiyetlere göre product line para harcama eğilimi:")
total_spend = df.groupby(['Gender', 'Product line'])['Total'].sum().reset_index()
gender_total_spend = df.groupby('Gender')['Total'].sum().reset_index()
spending_tendency = total_spend.merge(gender_total_spend, on='Gender')
spending_tendency['Spending tendency'] = spending_tendency['Total_x'] / spending_tendency['Total_y']
female_spending_tendency = spending_tendency[spending_tendency['Gender'] == 'Female']
female_spending_tendency = female_spending_tendency.sort_values(by='Spending tendency', ascending=False)
male_spending_tendency = spending_tendency[spending_tendency['Gender'] == 'Male']
male_spending_tendency = male_spending_tendency.sort_values(by='Spending tendency', ascending=False)
print("Erkekler")
print(male_spending_tendency)
print("---------------------")
print("Kadınlar")
print(female_spending_tendency)


Ülke genelindeki cinsiyetlere göre product line para harcama eğilimi:
Erkekler
   Gender            Product line     Total_x     Total_y  Spending tendency
9    Male       Health and beauty  30632.7525  155083.824           0.197524
6    Male  Electronic accessories  27235.5090  155083.824           0.175618
11   Male       Sports and travel  26548.1055  155083.824           0.171186
7    Male     Fashion accessories  23868.4950  155083.824           0.153907
10   Male      Home and lifestyle  23825.0355  155083.824           0.153627
8    Male      Food and beverages  22973.9265  155083.824           0.148139
---------------------
Kadınlar
   Gender            Product line     Total_x     Total_y  Spending tendency
2  Female      Food and beverages  33170.9175  167882.925           0.197584
1  Female     Fashion accessories  30437.4000  167882.925           0.181301
4  Female      Home and lifestyle  30036.8775  167882.925           0.178916
5  Female       Sports and travel  28574.72

* ### Ülke genelinde kadın ve erkeklerin para harcama eğilimleri kategorilere göre yukarda sıralanmıştır.

### Yangan Şehri

In [83]:
yangon_df = df[df['City'] == 'Yangon']
yangon_product_line_counts = yangon_df['Product line'].value_counts(normalize=True).sort_values(ascending=False)
print("Yangan şehrinde product line satın alma eğilimi:")
print(yangon_product_line_counts)
print("\n")

Yangan şehrinde product line satın alma eğilimi:
Product line
Home and lifestyle        0.191176
Electronic accessories    0.176471
Sports and travel         0.173529
Food and beverages        0.170588
Fashion accessories       0.150000
Health and beauty         0.138235
Name: proportion, dtype: float64




In [86]:
yangon_product_line_by_gender = yangon_df.groupby('Gender')['Product line'].value_counts(normalize=True)
print("Yangan şehrinde Cinsiyete göre product line satın alma eğilimi:")
for gender in ['Female', 'Male']:
    print(f"\n{gender}:")
    print(yangon_product_line_by_gender[gender].sort_values(ascending=False))
print("\n")


Yangan şehrinde Cinsiyete göre product line satın alma eğilimi:

Female:
Product line
Home and lifestyle        0.198758
Sports and travel         0.180124
Electronic accessories    0.173913
Fashion accessories       0.173913
Food and beverages        0.142857
Health and beauty         0.130435
Name: proportion, dtype: float64

Male:
Product line
Food and beverages        0.195531
Home and lifestyle        0.184358
Electronic accessories    0.178771
Sports and travel         0.167598
Health and beauty         0.145251
Fashion accessories       0.128492
Name: proportion, dtype: float64




In [82]:
yangon_spending_per_product_line = yangon_df.groupby('Product line')['Total'].sum().sort_values(ascending=False)
yangon_spending_per_product_line /= yangon_spending_per_product_line.sum()
print("Yangan şehrinde product line para harcama eğilimi:")
print(yangon_spending_per_product_line)
print("\n")

Yangan şehrinde product line para harcama eğilimi:
Product line
Home and lifestyle        0.211084
Sports and travel         0.182416
Electronic accessories    0.172477
Food and beverages        0.161611
Fashion accessories       0.153790
Health and beauty         0.118622
Name: Total, dtype: float64




In [87]:
yangon_spending_by_gender = yangon_df.groupby(['Gender', 'Product line'])['Total'].sum()
yangon_spending_by_gender = yangon_spending_by_gender.groupby(level=0).apply(lambda x: x / x.sum())
print("Yangan şehrinde cinsiyete göre product line para harcama eğilimi:")
for gender in ['Female', 'Male']:
    print(f"\n{gender}:")
    print(yangon_spending_by_gender[gender].sort_values(ascending=False))

Yangan şehrinde cinsiyete göre product line para harcama eğilimi:

Female:
Gender  Product line          
Female  Home and lifestyle        0.237226
        Electronic accessories    0.187093
        Fashion accessories       0.184651
        Sports and travel         0.152343
        Food and beverages        0.131404
        Health and beauty         0.107283
Name: Total, dtype: float64

Male:
Gender  Product line          
Male    Sports and travel         0.212682
        Food and beverages        0.192010
        Home and lifestyle        0.184775
        Electronic accessories    0.157767
        Health and beauty         0.130034
        Fashion accessories       0.122731
Name: Total, dtype: float64


### Naypyitaw Şehri

In [88]:
naypyitaw_df = df[df['City'] == 'Naypyitaw']

In [89]:
naypyitaw_product_line = naypyitaw_df['Product line'].value_counts(normalize=True).sort_values(ascending=False)
print("Naypyitaw şehrinde product line satın alma eğilimi:")
print(naypyitaw_product_line)
print("\n")

Naypyitaw şehrinde product line satın alma eğilimi:
Product line
Food and beverages        0.201220
Fashion accessories       0.198171
Electronic accessories    0.167683
Health and beauty         0.158537
Home and lifestyle        0.137195
Sports and travel         0.137195
Name: proportion, dtype: float64




In [90]:
naypyitaw_product_line_by_gender = naypyitaw_df.groupby('Gender')['Product line'].value_counts(normalize=True)
print("Naypyitaw şehrinde Cinsiyete göre product line satın alma eğilimi:")
for gender in ['Female', 'Male']:
    print(f"\n{gender}:")
    print(naypyitaw_product_line_by_gender[gender].sort_values(ascending=False))
print("\n")

Naypyitaw şehrinde Cinsiyete göre product line satın alma eğilimi:

Female:
Product line
Food and beverages        0.213483
Fashion accessories       0.196629
Sports and travel         0.162921
Electronic accessories    0.157303
Home and lifestyle        0.140449
Health and beauty         0.129213
Name: proportion, dtype: float64

Male:
Product line
Fashion accessories       0.200000
Health and beauty         0.193333
Food and beverages        0.186667
Electronic accessories    0.180000
Home and lifestyle        0.133333
Sports and travel         0.106667
Name: proportion, dtype: float64




In [91]:
naypyitaw_spending = naypyitaw_df.groupby('Product line')['Total'].sum().sort_values(ascending=False)
naypyitaw_spending /= naypyitaw_spending.sum()
print("Naypyitaw şehrinde prodcut line para harcama eğilimi:")
print(naypyitaw_spending)
print("\n")

Naypyitaw şehrinde prodcut line para harcama eğilimi:
Product line
Food and beverages        0.214951
Fashion accessories       0.194993
Electronic accessories    0.171558
Health and beauty         0.150272
Sports and travel         0.142553
Home and lifestyle        0.125673
Name: Total, dtype: float64




In [92]:
naypyitaw_spending_by_gender = naypyitaw_df.groupby(['Gender', 'Product line'])['Total'].sum()
naypyitaw_spending_by_gender = naypyitaw_spending_by_gender.groupby(level=0).apply(lambda x: x / x.sum())
print("Naypyitaw şehrinde cinsiyete göre product line para harcama eğilimi:")
for gender in ['Female', 'Male']:
    print(f"\n{gender}:")
    print(naypyitaw_spending_by_gender[gender].sort_values(ascending=False))

Naypyitaw şehrinde cinsiyete göre product line para harcama eğilimi:

Female:
Gender  Product line          
Female  Food and beverages        0.253887
        Fashion accessories       0.186749
        Sports and travel         0.182324
        Electronic accessories    0.145402
        Home and lifestyle        0.127144
        Health and beauty         0.104494
Name: Total, dtype: float64

Male:
Gender  Product line          
Male    Health and beauty         0.208038
        Fashion accessories       0.205395
        Electronic accessories    0.204564
        Food and beverages        0.165817
        Home and lifestyle        0.123818
        Sports and travel         0.092367
Name: Total, dtype: float64


### Mandalay Şehri

In [93]:
mandalay_df = df[df['City'] == 'Mandalay']

In [94]:
mandalay_product_line = mandalay_df['Product line'].value_counts(normalize=True).sort_values(ascending=False)
print("Mandalay şehrinde product line satın alma eğilimi:")
print(mandalay_product_line)
print("\n")

Mandalay şehrinde product line satın alma eğilimi:
Product line
Fashion accessories       0.186747
Sports and travel         0.186747
Electronic accessories    0.165663
Health and beauty         0.159639
Food and beverages        0.150602
Home and lifestyle        0.150602
Name: proportion, dtype: float64




In [95]:
mandalay_product_line_by_gender = mandalay_df.groupby('Gender')['Product line'].value_counts(normalize=True)
print("Mandalay şehrinde Cinsiyete göre product line satın alma eğilimi:")
for gender in ['Female', 'Male']:
    print(f"\n{gender}:")
    print(mandalay_product_line_by_gender[gender].sort_values(ascending=False))
print("\n")

Mandalay şehrinde Cinsiyete göre product line satın alma eğilimi:

Female:
Product line
Fashion accessories       0.203704
Sports and travel         0.185185
Food and beverages        0.179012
Electronic accessories    0.172840
Home and lifestyle        0.135802
Health and beauty         0.123457
Name: proportion, dtype: float64

Male:
Product line
Health and beauty         0.194118
Sports and travel         0.188235
Fashion accessories       0.170588
Home and lifestyle        0.164706
Electronic accessories    0.158824
Food and beverages        0.123529
Name: proportion, dtype: float64




In [96]:
mandalay_spending = mandalay_df.groupby('Product line')['Total'].sum().sort_values(ascending=False)
mandalay_spending /= mandalay_spending.sum()
print("Mandalay şehrinde prodcut line para harcama eğilimi:")
print(mandalay_spending)
print("\n")

Mandalay şehrinde prodcut line para harcama eğilimi:
Product line
Sports and travel         0.188217
Health and beauty         0.188146
Home and lifestyle        0.165250
Electronic accessories    0.160563
Fashion accessories       0.154554
Food and beverages        0.143270
Name: Total, dtype: float64




In [97]:
mandalay_spending_by_gender = mandalay_df.groupby(['Gender', 'Product line'])['Total'].sum()
mandalay_spending_by_gender = mandalay_spending_by_gender.groupby(level=0).apply(lambda x: x / x.sum())
print("Mandalay şehrinde cinsiyete göre product line para harcama eğilimi:")
for gender in ['Female', 'Male']:
    print(f"\n{gender}:")
    print(mandalay_spending_by_gender[gender].sort_values(ascending=False))

Mandalay şehrinde cinsiyete göre product line para harcama eğilimi:

Female:
Gender  Product line          
Female  Food and beverages        0.198570
        Home and lifestyle        0.180568
        Sports and travel         0.174062
        Fashion accessories       0.171581
        Electronic accessories    0.154294
        Health and beauty         0.120925
Name: Total, dtype: float64

Male:
Gender  Product line          
Male    Health and beauty         0.254937
        Sports and travel         0.202281
        Electronic accessories    0.166792
        Home and lifestyle        0.150031
        Fashion accessories       0.137637
        Food and beverages        0.088323
Name: Total, dtype: float64


Sonuç:

* ### Elimizdeki verilerin 3 farklı şehirden alındığı ve 6 farklı kategöriyi değerlendirdiğini anladık.
* ### Ülke genelinde Sırasıyla	%17,8
* ### 1.Fashion accessories 	%17,4
* ### 2.Food and beverages 	%17,0
* ### 3.Electronic accessories	%16,6
* ### 4.Sports and travel 	%16,0
* ### 5 Health and beauty 	%15,2

---
---


* ### Ülke genelinde kategorilere göre para harcama eğilimi:
* ### Food and beverages        0.173841
* ### Sports and travel         0.170676
* ### Electronic accessories    0.168245
* ### Fashion accessories       0.168147
* ### Home and lifestyle        0.166772
* ### Health and beauty         0.152318

---
---


* ### Şehirlere göre para harcama(satın alma eğilimi değil total harcanan paralara göre) Kategorilere göre eğiliminin sırası ile.
* ### Health and beauty için şehirlerdeki para harcama eğilimi:
* ### Mandalay     0.406163
* ### Naypyitaw    0.337753
* ### Yangon       0.256084

---


* ### Electronic accessories için şehirlerdeki para harcama eğilimi:
* ### City
* ### Naypyitaw    0.349095
* ### Yangon       0.337099
* ### Mandalay     0.313806
---

* ### Home and lifestyle için şehirlerdeki para harcama eğilimi:
* ### City
* ### Yangon       0.416198
* ### Mandalay     0.325818
* ### Naypyitaw    0.257985
---

* ### Sports and travel için şehirlerdeki para harcama eğilimi:
* ### City
* ### Mandalay     0.362612
* ### Yangon       0.351446
* ### Naypyitaw    0.285942
---


* ### Food and beverages için şehirlerdeki para harcama eğilimi:
* ### City
* ### Naypyitaw    0.423313
* ### Yangon       0.305693
* ### Mandalay     0.270994

---

* ### Fashion accessories için şehirlerdeki para harcama eğilimi:
* ### City
* ### Naypyitaw    0.397012
* ### Mandalay     0.302238
* ### Yangon       0.300750

---
---

* ### Ülke genelinde kadınların kategorilere göre satın alma eğilimleri:
* ### Fashion accessories     0.191617
* ### Food and beverages      0.179641
* ### Sports and travel       0.175649
* ### Electronic accessories  0.167665
* ### Home and lifestyle      0.157685
* ### Health and beauty       0.127745
---

* ### Ülke genelinde erkeklerin kategorilere göre satın alma eğilimleri:
* ### Health and beauty       0.176353
* ### Electronic accessories  0.172345
* ### Food and beverages      0.168337
* ### Fashion accessories     0.164329
* ### Home and lifestyle      0.162325
* ### Sports and travel       0.156313
---
---

* ### Ülke genelindeki cinsiyetlere göre product line para harcama eğilimi:
* ###    Erkekler
* ###    Gender            Product line     Total_x     Total_y  Spending tendency
* ###     Male       Health and beauty  30632.7525  155083.824           0.197524
* ###     Male  Electronic accessories  27235.5090  155083.824           0.175618
* ###     Male       Sports and travel  26548.1055  155083.824           0.171186
* ###     Male     Fashion accessories  23868.4950  155083.824           0.153907
* ###     Male      Home and lifestyle  23825.0355  155083.824           0.153627
* ###     Male      Food and beverages  22973.9265  155083.824           0.148139
---

* ###     Kadınlar
* ###     Gender            Product line     Total_x     Total_y  Spending tendency
* ###     Female      Food and beverages  33170.9175  167882.925           0.197584
* ###     Female     Fashion accessories  30437.4000  167882.925           0.181301
* ###     Female      Home and lifestyle  30036.8775  167882.925           0.178916
* ###     Female       Sports and travel  28574.7210  167882.925           0.170206
* ###     Female  Electronic accessories  27102.0225  167882.925           0.161434
* ###     Female       Health and beauty  18560.9865  167882.925           0.110559

---
---

* ### Yangan şehrinde product line satın alma eğilimi:
* ### Product line
* ### Home and lifestyle        0.191176
* ### Electronic accessories    0.176471
* ### Sports and travel         0.173529
* ### Food and beverages        0.170588
* ### Fashion accessories       0.150000
* ### Health and beauty         0.138235  
---

* ### Yangan şehrinde Cinsiyete göre product line satın alma eğilimi:

* ### Female:
* ### Product line
* ### Home and lifestyle        0.198758
* ### Sports and travel         0.180124
* ### Electronic accessories    0.173913
* ### Fashion accessories       0.173913
* ### Food and beverages        0.142857
* ### Health and beauty         0.130435
---

* ### Male:
* ### Product line
* ### Food and beverages        0.195531
* ### Home and lifestyle        0.184358
* ### Electronic accessories    0.178771
* ### Sports and travel         0.167598
* ### Health and beauty         0.145251
* ### Fashion accessories       0.128492

* ### Yangan şehrinde product line para harcama eğilimi:
* ### Product line
* ### Home and lifestyle        0.211084
* ### Sports and travel         0.182416
* ### Electronic accessories    0.172477
* ### Food and beverages        0.161611
* ### Fashion accessories       0.153790
* ### Health and beauty         0.118622

---

* ### Yangan şehrinde cinsiyete göre product line para harcama eğilimi:

* ### Female:
* ### Gender  Product line          
* ### Electronic accessories    0.187093
* ### Fashion accessories       0.184651
* ### Sports and travel         0.152343
* ### Food and beverages        0.131404
* ### Health and beauty         0.107283


* ### Male:
* ### Gender  Product line          
* ### Male    Sports and travel         0.212682
* ### Food and beverages        0.192010
* ### Home and lifestyle        0.184775
* ### Electronic accessories    0.157767
* ### Health and beauty         0.130034
* ### Fashion accessories       0.122731
---
---


* ### Naypyitaw şehrinde product line satın alma eğilimi:
* ### Product line
* ### Food and beverages        0.201220
* ### Fashion accessories       0.198171
* ### Electronic accessories    0.167683
* ### Health and beauty         0.158537
* ### Home and lifestyle        0.137195
* ### Sports and travel         0.137195

---

* ### Naypyitaw şehrinde Cinsiyete göre product line satın alma eğilimi:

* ### Female:
* ### Product line
* ### Food and beverages        0.213483
* ### Fashion accessories       0.196629
* ### Sports and travel         0.162921
* ### Electronic accessories    0.157303
* ### Home and lifestyle        0.140449
* ### Health and beauty         0.129213


* ### Male:
* ### Product line
* ### Fashion accessories       0.200000
* ### Health and beauty         0.193333
* ### Food and beverages        0.186667
* ### Electronic accessories    0.180000
* ### Home and lifestyle        0.133333
* ### Sports and travel         0.106667

---

* ### Naypyitaw şehrinde prodcut line para harcama eğilimi:
* ### Product line
* ### Food and beverages        0.214951
* ### Fashion accessories       0.194993
* ### Electronic accessories    0.171558
* ### Health and beauty         0.150272
* ### Sports and travel         0.142553
* ### Home and lifestyle        0.125673

---

* ### Naypyitaw şehrinde cinsiyete göre product line para harcama eğilimi:

* ### Female:
* ### Gender  Product line          
* ### Female  Food and beverages        0.253887
* ###         Fashion accessories       0.186749
* ###         Sports and travel         0.182324
* ###         Electronic accessories    0.145402
* ###        Home and lifestyle        0.127144
* ###         Health and beauty         0.104494

* ### Male:
* ### Gender  Product line          
* ### Male    Health and beauty         0.208038
* ###         Fashion accessories       0.205395
* ###         Electronic accessories    0.204564
* ###         Food and beverages        0.165817
* ###         Home and lifestyle        0.123818
* ###         Sports and travel         0.092367

---
---

* ### Mandalay şehrinde product line satın alma eğilimi:
* ### Product line
* ### Fashion accessories       0.186747
* ### Sports and travel         0.186747
* ### Electronic accessories    0.165663
* ### Health and beauty         0.159639
* ### Food and beverages        0.150602
* ### Home and lifestyle        0.150602
---


* ### Mandalay şehrinde Cinsiyete göre product line satın alma eğilimi:

* ### Female:
* ### Product line
* ### Fashion accessories       0.203704
* ### Sports and travel         0.185185
* ### Food and beverages        0.179012
* ### Electronic accessories    0.172840
* ### Home and lifestyle        0.135802
* ### Health and beauty         0.123457



* ### Male:
* ### Product line
* ### Health and beauty         0.194118
* ### Sports and travel         0.188235
* ### Fashion accessories       0.170588
* ### Home and lifestyle        0.164706
* ### Electronic accessories    0.158824
* ### Food and beverages        0.123529


* ### Mandalay şehrinde prodcut line para harcama eğilimi:
* ### Product line
* ### Sports and travel         0.188217
* ### Health and beauty         0.188146
* ### Home and lifestyle        0.165250
* ### Electronic accessories    0.160563
* ### Fashion accessories       0.154554
* ### Food and beverages        0.143270
---


* ### Mandalay şehrinde cinsiyete göre product line para harcama eğilimi:

* ### Female:
* ### Gender  Product line          
* ### Female  Food and beverages        0.198570
* ###         Home and lifestyle        0.180568
* ###         Sports and travel         0.174062
* ###         Fashion accessories       0.171581
* ###         Electronic accessories    0.154294
* ###         Health and beauty         0.120925

* ### Male:
* ### Gender  Product line          
* ### Male    Health and beauty         0.254937
* ###         Sports and travel         0.202281
* ###         Electronic accessories    0.166792
* ###         Home and lifestyle        0.150031
* ###         Fashion accessories       0.137637
* ###         Food and beverages        0.088323
---
---
